In [1]:
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

In [3]:
dPath = Path("../docs/dumps")

In [4]:
import pickle

In [5]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [6]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [7]:
with open(dPath / "not_running.pkl", 'rb') as filename:
    not_running = pickle.load(filename)

In [8]:
X_train = train_data.drop("Detected", axis=1)

In [9]:
y_train = train_data.Detected

In [10]:
X_valid = valid_data.drop("Detected", axis=1)

In [11]:
y_valid = valid_data.Detected

In [12]:
with open(dPath / "rf_exp_05_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [13]:
X_train = X_train[names] 

In [14]:
X_valid = X_valid[names]

In [15]:
X_train.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,cchalsteadCumulativeVolume,ppRVF,ppnumberOfMethods,...,cchalsteadEffort,ccmaintainabilityIndexNC,mmhalsteadVocabulary,pptcc,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmhalsteadEffort,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,1780.554266,13.0,16.0,...,32644.005473,99.687474,21.0,34.0,1.0,0.666667,5.0,2782.972319,3.0,4.0


In [16]:
from imblearn.over_sampling import ADASYN

In [17]:
sm = ADASYN(random_state=42, n_jobs=-1,  n_neighbors=5)

In [18]:
%time X_res, y_res = sm.fit_resample(X_train, y_train)

Wall time: 51.4 s


In [19]:
X_train_not_running = not_running.drop("Detected", axis=1)

In [20]:
y_train_not_running = not_running.Detected

In [21]:
X_train_not_running = X_train_not_running[names]

In [22]:
X_res = pd.concat([X_res,X_train_not_running, X_valid], ignore_index=True,axis=0)

In [23]:
y_res = pd.concat([y_res,y_train_not_running, y_valid], ignore_index=True,axis=0)

In [24]:
y_res.sum()/y_res.shape[0]

0.48736511651307735

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rf = RandomForestClassifier(n_estimators=100, max_features=0.7, oob_score=True, random_state=42, n_jobs=-1)

In [27]:
%time rf.fit(X_res, y_res)

Wall time: 12min 23s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.7,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [28]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [29]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid), roc_auc_score(y_valid, m.predict(X_valid))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [30]:
def conf_matr(m):
    y_pred = m.predict(X_valid)
    print(classification_report(y_valid, y_pred))

In [31]:
print_score(rf)

[0.9084503699953725, 0.920780490695745, 0.8785701137529768, 0.8628991361725975]


In [32]:
conf_matr(rf)

              precision    recall  f1-score   support

       False       0.87      0.80      0.83     16657
        True       0.93      0.96      0.95     50839

    accuracy                           0.92     67496
   macro avg       0.90      0.88      0.89     67496
weighted avg       0.92      0.92      0.92     67496



In [33]:
with open(dPath / "rf_exp_07.pkl", 'wb') as filename:
    pickle.dump(rf,filename)

In [34]:
names = X_train.columns

In [35]:
feat_importances = pd.Series(rf.feature_importances_, index=names)
feat_importances.sort_values(ascending=False,inplace=True)

In [36]:
feat_importances

numExecuted                      0.155936
MutatorClass                     0.098019
numAssertInTC                    0.063708
numTestCover                     0.051043
numAssertInTM                    0.049790
ppavcc                           0.036795
mmhalsteadDifficulty             0.029759
mmnumberOfExpressions            0.028413
mmhalsteadVocabulary             0.024578
mmhalsteadEffort                 0.024119
mmexternalMethodsCalled          0.024053
mminstanceVariablesReferenced    0.023697
mmvariablesReferenced            0.021888
ppmaxcc                          0.021261
ppmaintainabilityIndexNC         0.021014
ccmaintainabilityIndex           0.019492
returnType                       0.018810
ppmaintainabilityIndex           0.018367
ccexternalMethodCalls            0.018339
ccmaintainabilityIndexNC         0.017843
ccimportedPackages               0.015100
ccunweightedClassSize            0.014958
ppnumberOfClasses                0.014757
ccfanIn                          0

In [37]:
with open(dPath / "rf_exp_07_names.pkl", 'wb') as filename:
    pickle.dump(names,filename)